In [21]:
#Example 
n = 2
p = 2
X = np.random.normal(0, 1, (n, p))
y = np.random.normal(0, 1, n)
X_transpose = X.T
beta= [1,2]
beta_init= [1,2]
zeta_init = 6
eta_init =1

j=1
eta = [1,2]
sigma_sq=1
num_iterations=1000
nu=2

In [10]:
import numpy as np

In [12]:
# Fonction pour calculer mu_j
def compute_mu_j(beta, X, y, eta, sigma_sq, j):
    eta_j=eta[j]
    n = X.shape[0]
    mu_j = np.sum(X[:, j] * (y - np.dot(X, beta) + X[:, j] * beta[j])) / (X[:, j].dot(X[:, j]) / eta_j + 1 / sigma_sq)
    return mu_j

print(compute_mu_j(beta, X, y, eta, sigma_sq, j))

-0.5740237589254495


In [13]:
# Fonction pour calculer sigma_j^2
def compute_sigma_j_sq(X, eta, sigma_sq, j):
    eta_j=eta[j]
    sigma_j_sq = 1 / (X[:, j].dot(X[:, j]) / eta_j + 1 / sigma_sq)
    return sigma_j_sq

print(compute_sigma_j_sq(X, eta, sigma_sq, j))

0.4223769858205402


In [27]:
import numpy as np

def eta_t_plus_1(eta_sampled, beta_t, sigma_sq, zeta_t, nu):
    """
    Calcul de la densité a posteriori conditionnelle de eta_t+1.

    Arguments :
    eta_sampled : Valeurs échantillonnées de eta_t+1.
    beta_t : Vecteur de coefficients beta_t.
    sigma_sq : Variance sigma_t^2.
    zeta_t : Valeur zeta_t.
    nu : Paramètre nu.

    Returns :
    posterior_density : Densité a posteriori conditionnelle de eta_t+1.
    """
    p = len(eta_sampled)
    posterior_density = 1

    for j in range(p):
        m_tj = zeta_t * beta_t[j]**2 / (2 * sigma_sq)
        # Terme exponentiel
        exp_term = np.exp(-m_tj * eta_sampled[j])
        # Terme de normalisation
        normalization_term = eta_sampled[j]**((1 - nu) / 2) * (1 + nu * eta_sampled[j])**(nu + 1)
        # Mise à jour de la densité a posteriori conditionnelle
        posterior_density *= exp_term / normalization_term

    return posterior_density

# Exemple d'utilisation
eta_sampled = np.array([1, 2, 3])  # Exemple de valeurs échantillonnées de eta_t+1
beta_t = np.array([0.5, 0.8, 1.2])  # Exemple de coefficients beta_t
sigma_sq = 0.5  # Exemple de variance sigma_t^2
zeta_t = 1.5  # Exemple de valeur zeta_t
nu = 2  # Exemple de paramètre nu

posterior_density = eta_t_plus_1(eta_sampled, beta_t, sigma_sq, zeta_t, nu)
print("Densité a posteriori conditionnelle de eta_t+1 :", posterior_density)


Densité a posteriori conditionnelle de eta_t+1 : 3.2701949932084803e-10


In [36]:
u = np.random.uniform(0,4)
print(u)

2.780519320552555


In [39]:
import numpy as np

def slice_sampling_componentwise(eta_initial, beta_t, sigma_sq, zeta_t, nu, num_samples, burn_in=1000):
    """
    Échantillonne des valeurs de eta_t+1 en utilisant la méthode de slice sampling.

    Arguments :
    eta_initial : Valeur initiale de eta_t+1.
    beta_t : Vecteur de coefficients beta_t.
    sigma_sq : Variance sigma_t^2.
    zeta_t : Valeur zeta_t.
    nu : Paramètre nu.
    num_samples : Nombre total d'échantillons à générer.
    burn_in : Nombre d'itérations de brûlure.

    Returns :
    sampled_eta : Échantillons de eta_t+1.
    """
    # Initialisation
    sampled_eta = np.zeros((num_samples, len(eta_initial)))
    sampled_eta[0] = eta_initial
    
    # Itérations
    for i in range(1, num_samples):
        for j in range(len(eta_initial)):
            # Échantillonnage de la composante j de eta_t+1 conditionnellement aux autres composantes
            eta_j = eta_initial.copy()
            eta_j[j] = np.random.uniform(0, 10)  # Initialisation arbitraire pour la tranche
            
            # Définition de la fonction de densité conditionnelle pour la composante j
            def conditional_density(eta_j):
                m_tj = zeta_t * beta_t[j]**2 / (2 * sigma_sq)
                # Densité a posteriori conditionnelle de la composante j
                posterior_density_j = np.exp(-m_tj * eta_j) / (1 + nu * eta_j)**((nu + 1) / 2)
                return posterior_density_j
            
            # Slice sampling pour échantillonner une nouvelle valeur pour la composante j
            eta_j = slice_sampling(eta_j, conditional_density)
            sampled_eta[i, j] = eta_j[j]
        
    return sampled_eta[burn_in:]

def slice_sampling(eta, conditional_density, step_out_width=100, max_steps_out=1000):
    """
    Échantillonne une nouvelle valeur pour une composante en utilisant la méthode de slice sampling.

    Arguments :
    eta : Valeur initiale de la composante.
    conditional_density : Fonction de densité conditionnelle.
    step_out_width : Largeur initiale de la tranche.
    max_steps_out : Nombre maximal d'étapes pour élargir la tranche.

    Returns :
    new_eta : Nouvelle valeur échantillonnée.
    """
    # Sélection aléatoire de la hauteur de la tranche
    u = np.random.uniform(0, conditional_density(eta))
    print(u)
    
    # Détermination de la tranche initiale
    L = eta - np.random.uniform(0, step_out_width)
    R = L + step_out_width
    
    # Échantillonnage
    for _ in range(max_steps_out):
        # Échantillonnage d'une proposition dans la tranche
        eta_proposal = np.random.uniform(L, R)
        
        # Calcul de la densité conditionnelle pour eta_proposal
        density_value = conditional_density(eta_proposal)
        
        # Acceptation/rejet de la proposition
        if u < density_value:
            return eta_proposal
        
        # Mise à jour de la tranche
        if eta_proposal < eta:
            L = eta_proposal
        else:
            R = eta_proposal
    
    return eta  # Retourne la valeur initiale si le nombre maximal d'étapes est atteint


# Exemple d'utilisation
eta_initial = np.array([1, 2, 3])  # Valeur initiale de eta_t+1
beta_t = np.array([0.5, 0.8, 1.2])  # Exemple de coefficients beta_t
sigma_sq = 0.5  # Exemple de variance sigma_t^2
zeta_t = 1.5  # Exemple de valeur zeta_t
nu = 0.1  # Exemple de paramètre nu
num_samples = 10000  # Nombre total d'échantillons à générer

sampled_eta = slice_sampling_componentwise(eta_initial, beta_t, sigma_sq, zeta_t, nu, num_samples)
print("Échantillons de eta_t+1 :", sampled_eta)


[0.01550373 0.05603989 0.22884044]


/tmp/ipykernel_5360/3773283400.py:34: RuntimeWarning: invalid value encountered in power
  posterior_density_j = np.exp(-m_tj * eta_j) / (1 + nu * eta_j)**((nu + 1) / 2)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [29]:
# il faut faire une fonction calculate_likelihood ici 

def calculate_likelihood(X, y, beta_t, zeta_t_plus_1, eta_t_plus_1):    # il faut que les béta_sampled et les sigma_sq_sampled existe pour le faire.
    """
    Calcule la vraisemblance de y sachant zeta_t+1 et eta_t+1.

    Arguments :
    y : Vecteur des observations.
    X : Matrice des prédicteurs.
    beta_t : Échantillons de beta à t.
    sigma_sq_samples : Échantillons de sigma_sq.
    zeta : Valeur de zeta_t+1.
    eta : Vecteur des valeurs eta_t+1.

    Returns :
    likelihood : La vraisemblance de y sachant zeta_t+1 et eta_t+1.
    """
    n_samples = len(beta_t)
    likelihood = 0

    for i in range(n_samples):
        beta = beta_t[i]
        sigma_sq = sigma_sq_samples[i]
        
        # Calcul de la densité conditionnelle de y sachant beta et sigma_sq
        data_density = np.exp(-0.5 * n * np.log(2 * np.pi * sigma_sq) - 0.5 * np.linalg.slogdet(Sigma)[1] \
                     - 0.5 * np.dot(np.dot((y - np.dot(X.T, beta)).T, np.linalg.inv(Sigma)), (y - np.dot(X.T, beta))))
        
        # Calcul de la densité jointe de beta et sigma_sq sachant zeta_t+1 et eta_t+1
        joint_prior = np.exp(-0.5 * np.sum(beta**2) / (zeta_t_plus_1 * eta_t_plus_1) - 0.5 * np.log(sigma_sq))
        
        # Ajout de la contribution de cet échantillon à la vraisemblance
        likelihood += data_density * joint_prior
    
    # Moyenne des contributions de tous les échantillons
    likelihood /= n_samples
    
    return likelihood



def sample_zeta_t_plus_1(zeta_t, eta_t_plus_1, sigma_sq, sigma_mrth=0.8): 
# y, X, beta, sigma_sq ???
    
    """
    Échantillonne la valeur de zeta_t+1 conditionnellement à zeta_t et eta_t+1.

    Arguments :
    zeta_t : Valeur zeta_t.
    eta_t_plus_1 : Valeur échantillonnée de eta_t+1.
    y : Vecteur de données.
    X : Matrice de design.
    beta : Vecteur de coefficients beta_t.
    sigma_sq : Variance sigma_t^2.
    sigma_mrth : Écart-type de la proposition normale = 0.8 selon l'article.

    Returns :
    zeta_sampled : Valeur échantillonnée de zeta_t+1.
    """
    zeta_proposed = zeta_t
    # Calcul de la matrice Sigma
    Sigma = np.dot(X, X.T) + zeta_proposed * np.diag(eta_t_plus_1)

    # Proposition d'un nouvel échantillon de log(zeta_t+1)
    log_zeta_proposed = np.random.normal(np.log(zeta_t), sigma_mrth)

    # Calcul des termes de probabilité a priori
    prior_current = -0.5 * zeta_t ** 2
    prior_proposed = -0.5 * np.exp(2 * log_zeta_proposed)

    # On calcule la vraisemblance conditionnelle des données
    log_likelihood = np.log(calculate_likelihood(X, y, beta_t, zeta_proposed, eta_t_plus_1)) # cf fonction calculate_likelihood au dessus
    
    # Log-probabilité du log-posterior pour les valeurs actuelles et proposées
    log_posterior_current = log_likelihood + prior_current
    log_posterior_proposed = log_likelihood + prior_proposed

    # Calcul du ratio de probabilité
    acceptance_ratio = np.exp(log_posterior_proposed - log_posterior_current)

    # Acceptation ou rejet de la proposition
    if np.random.uniform(0, 1) < acceptance_ratio:
        zeta_sampled = np.exp(log_zeta_proposed)
    else:
        zeta_sampled = zeta_t

    return zeta_sampled



In [ ]:
# Fonction pour l'échantillonnage de Gibbs
def gibbs_sampling(X, y, beta_init, eta_init, zeta_init, sigma_sq, num_iterations, nu):
    num_features = X.shape[1]
    zeta = zeta_init
    eta = eta_init
    # Boucle sur le nombre d'itérations
    for t in range(num_iterations):
        beta_new = beta_init
        eta = sample_eta_t_plus_1(beta_new, sigma_sq, zeta, nu)
        zeta = sample_zeta(zeta, eta, sigma_sq, sigma_mrth=0.8)
        
        # Boucle sur chaque coordonnée beta_j
        for j in range(num_features):
            mu_j = compute_mu_j(beta_new, X, y, eta, sigma_sq, j)
            sigma_j_sq = compute_sigma_j_sq(X, eta, sigma_sq, j)
            beta_new[j] = np.random.normal(mu_j, np.sqrt(sigma_j_sq))
        
        # Mettre à jour les coordonnées beta pour l'itération suivante
        beta_current = beta_new
    
    return beta_current

print(gibbs_sampling(X, y, beta_init, eta, sigma_sq, num_iterations))

NameError: name 'X' is not defined